# Testing AE 64

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from random import randint

from IPython.display import Image
from IPython.core.display import Image, display

%load_ext autoreload
%autoreload 2

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [4]:
class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

In [5]:
class autoencoder(nn.Module):
    def __init__(self,i=1024,o=64):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(512, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(1024, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            Flatten(),
            nn.Linear(in_features=i, out_features=o),
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=o, out_features=i),
            UnFlatten(),
            nn.ConvTranspose2d(1024, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2),
            nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 128, 128)
    return x

In [7]:
learning_rate = 1e-3
criterion = nn.MSELoss(reduction='sum')

In [8]:
from IPython.display import Image
from IPython.core.display import Image, display

In [9]:
test= autoencoder()
test.to(device)

autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    (16): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, t

In [10]:
test.load_state_dict(torch.load('./autoencoder_AE_Thesis_Pattern1_latent64.pth'))
test.eval()

autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    (16): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, t

In [11]:
bs = 32
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/N', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/N
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [12]:
dataloader_iterator = iter(test_dataloader)
for i in range(len(test_dataloader)):
    try:
        data, target = next(dataloader_iterator)
        save_image(data, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Original_AE_latent64_{}.png'.format(i))
    except StopIteration:
        dataloader_iterator = iter(dataloader)
        data, target = next(dataloader_iterator)

In [13]:
b=-1
for data in test_dataloader:
    img, _ = data
    img = Variable(img).cuda()
    # ===================forward=====================
    output = test(img)
    loss = criterion(output, img)
# ===================log========================
    pic = to_img(output.cpu().data)
    b=b+1
    save_image(pic, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Reconstructed_Normal_AE_latent64_{}.png'.format(b))
#     print('{:.4f}'.format(loss.data))

In [14]:
bs = 1
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/N', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/N
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [15]:
loss_append=[]
for data in test_dataloader:
    img, _ = data
    img = Variable(img).cuda()
    # ===================forward=====================
    output = test(img)
    loss = criterion(output, img)
# ===================log========================
    pic = to_img(output.cpu().data)
    loss = round(float(loss.data),4)
#     loss = loss.cpu()
#     loss = loss.detach().numpy()
    loss_append.append(loss)
#     print('{:.4f}'.format(loss.data))
#     loss_append.append='{:.4f}'.format(loss)
print(loss_append)

[13.3114, 17.3876, 15.817, 14.4219, 12.3543, 22.3369, 20.6648, 20.5269, 21.3785, 22.7464, 22.2325, 16.2996, 15.4987, 16.8616, 15.5455, 18.5303, 25.3443, 25.3821, 17.8025, 11.5602, 16.4387, 15.2584, 15.9959, 16.7715, 14.3889, 10.6258, 10.6984, 13.867, 15.4941, 13.2474, 20.6186, 16.1938, 14.6357, 24.135, 21.5189, 18.3244, 20.1778, 18.779, 21.0837, 29.5901, 10.3367, 8.4233, 15.3989, 16.8546, 17.6925, 14.5637, 31.7804, 33.2833, 32.9182, 40.4152, 42.0749, 40.0744, 34.897, 39.109, 48.2419, 52.5372, 63.0014, 63.8946, 71.3912, 98.7255, 76.1255, 83.1536, 92.683, 107.6207, 84.9552, 94.3247, 125.2037, 65.1659, 43.6537, 55.1431, 73.4702, 86.0429, 128.3539, 116.5527, 149.1821, 132.6072, 174.2661, 186.4011, 137.2105, 136.6839, 11.8329, 11.0797, 10.5122, 12.4967, 19.6701, 17.5804, 20.3997, 12.468, 9.2032, 7.0911, 7.7419, 10.6809, 10.9042, 12.0578, 12.8524, 11.3462, 9.7128, 8.9833, 12.1193, 18.5071, 10.8753, 8.728, 10.6808, 9.777, 9.1351, 7.3599, 11.2702, 12.3639, 13.9296, 16.8435, 20.4167, 18.8219, 1

In [16]:
import numpy as np
percentile=np.percentile(loss_append, 95, axis=0)
percentile

127.25132999999997

In [17]:
bs = 32
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/A', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/A
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [18]:
dataloader_iterator = iter(test_dataloader)
for i in range(len(test_dataloader)):
    try:
        data, target = next(dataloader_iterator)
        save_image(data, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Original_Abnormal_AE_latent64_{}.png'.format(i))
    except StopIteration:
        dataloader_iterator = iter(dataloader)
        data, target = next(dataloader_iterator)

In [19]:
b=-1
for data in test_dataloader:
    img, _ = data
    img = Variable(img).cuda()
    # ===================forward=====================
    output = test(img)
    loss = criterion(output, img)
# ===================log========================
    pic = to_img(output.cpu().data)
    b=b+1
    save_image(pic, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Reconstructed_Abnormal_AE_latent64_{}.png'.format(b))
#     print('{:.4f}'.format(loss.data))

In [20]:
bs = 1
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/A', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/A
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [21]:
loss_append_ano=[]
for data in test_dataloader:
    img, _ = data
    img = Variable(img).cuda()
    # ===================forward=====================
    output = test(img)
    loss = criterion(output, img)
# ===================log========================
    pic = to_img(output.cpu().data)
    loss = round(float(loss.data),4)
#     print('{:.4f}'.format(loss.data))
    loss_append_ano.append(loss)
print(loss_append_ano)

[112.2614, 89.3293, 123.2368, 111.7186, 189.2, 214.5542, 102.2892, 63.0394, 94.9869, 77.7289, 70.8234, 52.5199, 56.5777, 63.6536, 42.9297, 67.2392, 503.1303, 320.2472, 354.2481, 414.6165, 438.2722, 505.069, 327.9466, 286.1786, 25.7157, 248.2355, 10465.3428, 10321.1836, 8645.9795, 8409.1719, 9326.7109, 10761.7549, 13227.8584, 6137.2871, 4474.6812, 3114.7385, 3279.9946, 4472.8267, 8107.9795, 14104.8682, 5287.834, 3942.7998, 334.3032, 280.564, 297.4358, 66.3724, 79.2785, 78.0419, 53.66, 37.7688, 74.6163, 63.6924, 53.3421, 87.2596, 60.5847, 85.823, 105.4669, 176.8144, 67.8948, 82.3275, 31.9626, 75.7112, 41.5004, 39.7015, 2665.2939, 2374.4434, 1040.9232, 2491.7925, 1065.9443, 1685.6021, 1476.9156, 1715.5786, 142.2154, 138.5142, 458.9952, 687.2787, 862.9761, 841.7038, 864.0051, 152.4262, 4663.9375, 1049.4255, 4361.5215, 1175.4775, 3711.2673, 1321.6953, 1066.3696, 2455.8855, 2369.97, 2386.0391, 2215.9375, 2466.5295, 2387.9531, 2315.5498, 2330.7031, 351.7585, 356.4008, 249.9377, 41.1909, 137.2

In [22]:
#TP
TP=0
FN=0
for j in range(128):   
    if loss_append_ano[j]>percentile:   
        TP=TP+1
    else:
        FN=FN+1

TN=0
FP=0
for j in range(128):   
    if loss_append[j]>percentile:   
        FP=FP+1
    else:
        TN=TN+1
print ("TN=",TN,"TP=",TP)
print ("FP=",FP,"FN=",FN)

TN= 121 TP= 93
FP= 7 FN= 35


In [23]:
percentile=np.percentile(loss_append, 90, axis=0)
percentile
#TP
TP=0
FN=0
for j in range(128):   
    if loss_append_ano[j]>percentile:   
        TP=TP+1
    else:
        FN=FN+1

TN=0
FP=0
for j in range(128):   
    if loss_append[j]>percentile:   
        FP=FP+1
    else:
        TN=TN+1
print ("TN=",TN,"TP=",TP)
print ("FP=",FP,"FN=",FN)

TN= 115 TP= 101
FP= 13 FN= 27


In [24]:
normal_testing=loss_append
normal_testing = np.array_split(normal_testing, len(normal_testing)/32)

anomaly_testing=loss_append_ano
anomaly_testing = np.array_split(anomaly_testing, len(anomaly_testing)/32)

a=np.array(normal_testing).shape
a[0]

4

In [25]:
print("Normal Testing:")
for b in range(a[0]):
    print('\nNormal Batch{}'.format(b+1))
    for i in range(32):
        if (i+1)%8!=0:
            print(round(normal_testing[b][i],5),end=" ")
        else:
            print(round(normal_testing[b][i],5))

print("\nAnomaly Testing:")
for c in range(a[0]):
    print('\nAnomaly Batch{}'.format(c+1))
    for z in range(32):
        if (z+1)%8!=0:
            print(round(anomaly_testing[c][z],5),end=" ")
        else:
            print(round(anomaly_testing[c][z],5))
b=0
c=0
i=0
z=0

Normal Testing:

Normal Batch1
13.3114 17.3876 15.817 14.4219 12.3543 22.3369 20.6648 20.5269
21.3785 22.7464 22.2325 16.2996 15.4987 16.8616 15.5455 18.5303
25.3443 25.3821 17.8025 11.5602 16.4387 15.2584 15.9959 16.7715
14.3889 10.6258 10.6984 13.867 15.4941 13.2474 20.6186 16.1938

Normal Batch2
14.6357 24.135 21.5189 18.3244 20.1778 18.779 21.0837 29.5901
10.3367 8.4233 15.3989 16.8546 17.6925 14.5637 31.7804 33.2833
32.9182 40.4152 42.0749 40.0744 34.897 39.109 48.2419 52.5372
63.0014 63.8946 71.3912 98.7255 76.1255 83.1536 92.683 107.6207

Normal Batch3
84.9552 94.3247 125.2037 65.1659 43.6537 55.1431 73.4702 86.0429
128.3539 116.5527 149.1821 132.6072 174.2661 186.4011 137.2105 136.6839
11.8329 11.0797 10.5122 12.4967 19.6701 17.5804 20.3997 12.468
9.2032 7.0911 7.7419 10.6809 10.9042 12.0578 12.8524 11.3462

Normal Batch4
9.7128 8.9833 12.1193 18.5071 10.8753 8.728 10.6808 9.777
9.1351 7.3599 11.2702 12.3639 13.9296 16.8435 20.4167 18.8219
10.7058 8.6374 15.3429 9.6943 11.7062 